<a href="https://colab.research.google.com/github/sandovalmedeiros/PythonGeo/blob/main/ETL579_11583_Semana1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import classification_report

# Carregar os dados
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Função para criar features
def criar_features(X):
    subs = {'female':1, 'male':0}
    X['mulher'] = X['Sex'].replace(subs)

    X['Fare'] = X['Fare'].fillna(X['Fare'].mean())

    X['Age'] = X['Age'].fillna(X['Age'].mean())

    X['Embarked'] = X['Embarked'].fillna('S')

    subs = {'S':1, 'C':2, 'Q':3}
    X['porto'] = X['Embarked'].replace(subs)

    X['crianca'] = np.where(X['Age'] < 12, 1, 0)

    return X

# Aplicar a função aos dados de treino e teste
X_train = criar_features(train.drop(['PassengerId', 'Survived'], axis=1))
X_test = criar_features(test.drop(['PassengerId'], axis=1))

# Selecionar características
features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'mulher', 'porto', 'crianca']
X_train = X_train[features]
X_test = X_test[features]
y_train = train['Survived']

# Padronizar as variáveis
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

# Otimização de Hiperparâmetros para RandomForestClassifier
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=0), param_grid, cv=5)
grid_search.fit(X_train_sc, y_train)
best_params = grid_search.best_params_

# Treinamento do modelo final usando os melhores parâmetros
model_rf = RandomForestClassifier(**best_params, random_state=0)
model_rf.fit(X_train_sc, y_train)

# Avaliação do modelo
score = cross_val_score(model_rf, X_train_sc, y_train, cv=10)
print("Cross-validated Accuracy:", np.mean(score))

# Relatório de classificação
y_pred = model_rf.predict(X_train_sc)
print(classification_report(y_train, y_pred))

# Predição nos dados de teste
y_pred_test = model_rf.predict(X_test_sc)

# Criação do arquivo de submissão
submission = pd.DataFrame(test['PassengerId'])
submission['Survived'] = y_pred_test
submission.to_csv('submission_14_02_2024.csv', index=False)


FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'